## Merge spatial dataset with the worship centres and foreign distributions by municipalities

## THIS NOTEBOOK IS NEEDED TO RUN THE "Correlations_religion.ipynb" AND "Religion_distribution_map.ipynb" NOTEBOOKS !!!

In this notebook we will generate a file "foreign_faith_distribution.json" with the spatial information of the catalonian municipalities. This file will serve to plot the worship centres distribution map in Catalonia and do some correlations calculations related to the foreign inhabitants distribution.

In [36]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Load the spatial data

In [37]:
# Path to GeoJSON file
archivo_geojson = "divisions-administratives-v2r1-municipis-1000000-20240705.json"


# Load
map_data = gpd.read_file(archivo_geojson)

# Stablish 'geometry' column with the spatial information
map_data = map_data.set_geometry("geometry")

map_data.head(n=2)

,CODIMUNI,NOMMUNI,CAPMUNI,AREAM5000,CODICOMAR,NOMCOMAR,CAPCOMAR,CODIVEGUE,NOMVEGUE,CAPVEGUE,CODIPROV,NOMPROV,CAPPROV,geometry
0,080018,Abrera,Abrera,19.9781,11,Baix Llobregat,Sant Feliu de Llobregat,01,Barcelona,Barcelona,08,Barcelona,Barcelona,"MULTIPOLYGON (((1.92486 41.53663, 1.92197 41.5..."
1,080023,Aguilar de Segarra,Aguilar de Segarra,43.2198,07,Bages,Manresa,07,Catalunya Central,*,08,Barcelona,Barcelona,"MULTIPOLYGON (((1.61829 41.76885, 1.61278 41.7..."


In [38]:
population_data= pd.read_csv("census_migrants2021_utf8.csv", encoding="utf-8")
population_data.head(n=2)

,Categoria,(1) Población,Población extranjera. Total,Población extranjera. % vert.,Población extranjera. % sobre (1)
0,Barcelona,1636732,348302,27.85,21.28
1,"Hospitalet de Llobregat, l'",264657,58685,4.69,22.17


## Associate municipalities with codes

In [39]:
# Function to associates municipalities with their codes using the municipality_code.txt file
def cargar_asociacion(filepath):
    """
    Carga la asociación código-municipio desde un archivo .txt y la devuelve como un diccionario.
    """
    asociacion = {}
    with open(filepath, "r") as file:
        for line in file:
            codigo, municipality = line.strip().split(":")
            asociacion[municipality] = codigo
    return asociacion

# Load the codes
codigo_municipio_dict = cargar_asociacion("municipality_code.txt")

# Función para obtener el código de un municipio
def obtain_code(municipality):
    """
    Devuelve el código correspondiente al municipio proporcionado.
    Si el municipio no existe en la asociación, devuelve None.
    """
    return codigo_municipio_dict.get(municipality, None)

# Crear una nueva columna "Codigo" en population_data
population_data['Codigo'] = population_data['Categoria'].apply(obtain_code)

# Reorganizar las columnas para que "Codigo" sea la primera
population_data = population_data[['Codigo'] + [col for col in population_data.columns if col != 'Codigo']]

population_data.head(n=2)

,Codigo,Categoria,(1) Población,Población extranjera. Total,Población extranjera. % vert.,Población extranjera. % sobre (1)
0,08019,Barcelona,1636732,348302,27.85,21.28
1,08101,"Hospitalet de Llobregat, l'",264657,58685,4.69,22.17


In [40]:
# Truncar la columna CODIMUNI en map_data a 5 dígitos
map_data['Truncated_CODIMUNI'] = map_data['CODIMUNI'].astype(str).str[:5]

# Realizar el merge utilizando el código de 5 dígitos
merged_data = map_data.merge(
    population_data,
    left_on='Truncated_CODIMUNI',    # Columna truncada de map_data             
    right_on='Codigo',               # Columna de population_data
    how='left'                      # Merge solo con coincidencias
)

# Eliminar la columna auxiliar 'Truncated_CODIMUNI' si no es necesaria
merged_data.drop(columns=['Truncated_CODIMUNI'], inplace=True)

merged_data.head(n=2)

,CODIMUNI,NOMMUNI,CAPMUNI,AREAM5000,CODICOMAR,NOMCOMAR,CAPCOMAR,CODIVEGUE,NOMVEGUE,CAPVEGUE,CODIPROV,NOMPROV,CAPPROV,geometry,Codigo,Categoria,(1) Población,Población extranjera. Total,Población extranjera. % vert.,Población extranjera. % sobre (1)
0,080018,Abrera,Abrera,19.9781,11,Baix Llobregat,Sant Feliu de Llobregat,01,Barcelona,Barcelona,08,Barcelona,Barcelona,"MULTIPOLYGON (((1.92486 41.53663, 1.92197 41.5...",08001,Abrera,12620.0,842.0,0.07,6.67
1,080023,Aguilar de Segarra,Aguilar de Segarra,43.2198,07,Bages,Manresa,07,Catalunya Central,*,08,Barcelona,Barcelona,"MULTIPOLYGON (((1.61829 41.76885, 1.61278 41.7...",08002,Aguilar de Segarra,286.0,6.0,<0.01,2.10


## Merge faith data (with years)

In [41]:
# Cargar el archivo CSV
centres_data = pd.read_csv("ReligionCentres_CAT.csv")

# Inicializar el dataframe vacío para los resultados
centres_faith = pd.DataFrame()

# Definir las categorías de religiones principales
categories = {
    "Catholic": "Església catòlica",
    "Evangelical": "Esglésies Evangèliques",
    "Islam": "Islam"
}

# Definir las religiones que van a la categoría "Others"
others_categories = [
    "Sikhisme", "Judaisme", "Budisme", "Hinduisme", "Altres", 
    "Testimonis cristians de Jehovà", "Església Adventista del Setè Dia", 
    "Fe Bahà'í", "Esglésies orientals", "Església de Jesucrist dels Sants dels Darrers Dies", 
    "Taoisme", "Paganisme"
]

# Iterar por los años disponibles (2020, 2021, 2022)
for year in [2020, 2021, 2022]:
    # Filtrar los datos para el año actual
    year_data = centres_data[centres_data['ANY'] == year]
    
    # Agrupar por municipio y religión y contar
    religion_counts = year_data.groupby('CODI MUNICIPI')['CONFESSIÓ'].value_counts().unstack(fill_value=0)
    
    # Crear las columnas de la tabla final para este año
    year_columns = {
        "Code": religion_counts.index
    }
    
    # Para cada religión principal, asignar el conteo para el año actual
    for religion_key, religion_name in categories.items():
        # Usamos get() para manejar el caso de religiones que no aparecen en todos los municipios
        year_columns[f"{religion_key}_{year}"] = religion_counts.get(religion_name, 0)
    
    # Ahora, añadir las religiones de otros años (Others)
    # Iterar sobre las religiones "Others" y agregarlas al dataframe
    for other_religion in others_categories:
        year_columns[f"{other_religion}_{year}"] = religion_counts.get(other_religion, 0)

    # Agregar a la tabla de resultados para el año actual
    if centres_faith.empty:
        centres_faith = pd.DataFrame(year_columns)
    else:
        centres_faith = centres_faith.merge(
            pd.DataFrame(year_columns),
            on="Code",
            how="outer"
        )

# Mostrar los primeros registros
centres_faith.head(2)


,Code,Catholic_2020,Evangelical_2020,Islam_2020,Sikhisme_2020,Judaisme_2020,Budisme_2020,Hinduisme_2020,Altres_2020,Testimonis cristians de Jehovà_2020,...,Budisme_2022,Hinduisme_2022,Altres_2022,Testimonis cristians de Jehovà_2022,Església Adventista del Setè Dia_2022,Fe Bahà'í_2022,Esglésies orientals_2022,Església de Jesucrist dels Sants dels Darrers Dies_2022,Taoisme_2022,Paganisme_2022
0,80018,2,1,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,80023,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Truncar la derecha ahora

In [42]:
# Primero, crear la columna 'Truncated_CODIMUNI' sin usar un ciclo 'for'
# Si 'CODIMUNI' tiene 5 dígitos, truncar el primer dígito; si tiene 6, dejarlo intacto
merged_data['Truncated_CODIMUNI'] = merged_data['CODIMUNI'].apply(lambda x: str(x)[1:] if len(str(x)) == 5 else str(x))

# Asegúrate de que ambas columnas sean de tipo 'int'
merged_data['Truncated_CODIMUNI'] = merged_data['Truncated_CODIMUNI'].astype(int)
centres_faith['Code'] = centres_faith['Code'].astype(int)

# Realizar el merge utilizando el código de 5 dígitos
foreign_faith_distribution = merged_data.merge(
    centres_faith,
    left_on='Truncated_CODIMUNI',  # Columna truncada de merged_data
    right_on='Code',               # Columna de centres_faith
    how='left'                     # Merge solo con coincidencias
)

# Eliminar la columna auxiliar 'Truncated_CODIMUNI' si no es necesaria
foreign_faith_distribution.drop(columns=['Truncated_CODIMUNI'], inplace=True)

# Verificar el resultado
foreign_faith_distribution.head(n=2)


,CODIMUNI,NOMMUNI,CAPMUNI,AREAM5000,CODICOMAR,NOMCOMAR,CAPCOMAR,CODIVEGUE,NOMVEGUE,CAPVEGUE,...,Budisme_2022,Hinduisme_2022,Altres_2022,Testimonis cristians de Jehovà_2022,Església Adventista del Setè Dia_2022,Fe Bahà'í_2022,Esglésies orientals_2022,Església de Jesucrist dels Sants dels Darrers Dies_2022,Taoisme_2022,Paganisme_2022
0,080018,Abrera,Abrera,19.9781,11,Baix Llobregat,Sant Feliu de Llobregat,01,Barcelona,Barcelona,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,080023,Aguilar de Segarra,Aguilar de Segarra,43.2198,07,Bages,Manresa,07,Catalunya Central,*,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Create columns for new variables in order to represent information

We create new columns in the dataset with variables of interest. 

We normalize the foreign inhabitants and the nº of centres with a z-score normalization, in order to be able later on to calculate correlations (using the same orders of magnitudes)

In [43]:
#Others
foreign_faith_distribution['Others_2021'] = (
    foreign_faith_distribution['Sikhisme_2021'] +
    foreign_faith_distribution['Judaisme_2021'] +
    foreign_faith_distribution['Budisme_2021'] +
    foreign_faith_distribution['Hinduisme_2021'] +
    foreign_faith_distribution['Altres_2021'] +
    foreign_faith_distribution['Testimonis cristians de Jehovà_2021'] +
    foreign_faith_distribution['Església Adventista del Setè Dia_2021'] +
    foreign_faith_distribution['Fe Bahà\'í_2021'] +
    foreign_faith_distribution['Esglésies orientals_2021'] +
    foreign_faith_distribution['Església de Jesucrist dels Sants dels Darrers Dies_2021'] +
    foreign_faith_distribution['Taoisme_2021'] +
    foreign_faith_distribution['Paganisme_2021']
)

#Normalised 2022 centres (per population)
foreign_faith_distribution['Normalised_catholic_2021'] = foreign_faith_distribution['Catholic_2021']/foreign_faith_distribution['(1) Población']
foreign_faith_distribution['Normalised_evangelical_2021'] = foreign_faith_distribution['Evangelical_2021']/foreign_faith_distribution['(1) Población']
foreign_faith_distribution['Normalised_islam_2021'] = foreign_faith_distribution['Islam_2021']/foreign_faith_distribution['(1) Población']
foreign_faith_distribution['Normalised_others_2021'] = foreign_faith_distribution['Others_2021']/foreign_faith_distribution['(1) Población']

#Total number of centres per municipality
foreign_faith_distribution['Total_2021'] = (
    foreign_faith_distribution['Catholic_2021']+
    foreign_faith_distribution['Evangelical_2021']+
    foreign_faith_distribution['Islam_2021']+
    foreign_faith_distribution['Others_2021']
)
    
###########################################################################################
### FOR CORRELATIONS: Z-score normalization

# Z-score normalization
# Normalización Z-Score para la cantidad de centros religiosos
foreign_faith_distribution['Zscore_catholic_2021'] = (
    foreign_faith_distribution['Catholic_2021'] - foreign_faith_distribution['Catholic_2021'].mean()
) / foreign_faith_distribution['Catholic_2021'].std()

foreign_faith_distribution['Zscore_evangelical_2021'] = (
    foreign_faith_distribution['Evangelical_2021'] - foreign_faith_distribution['Evangelical_2021'].mean()
) / foreign_faith_distribution['Evangelical_2021'].std()

foreign_faith_distribution['Zscore_islam_2021'] = (
    foreign_faith_distribution['Islam_2021'] - foreign_faith_distribution['Islam_2021'].mean()
) / foreign_faith_distribution['Islam_2021'].std()

foreign_faith_distribution['Zscore_others_2021'] = (
    foreign_faith_distribution['Others_2021'] - foreign_faith_distribution['Others_2021'].mean()
) / foreign_faith_distribution['Others_2021'].std()

# Normalización Z-Score para la población extranjera "(1) Población"
foreign_faith_distribution['Zscore_foreign_2021'] = (
    foreign_faith_distribution['Población extranjera. Total'] - foreign_faith_distribution['Población extranjera. Total'].mean()
) / foreign_faith_distribution['Población extranjera. Total'].std()


## WE ADD A NEW ONE FOR THE SHANNON-WIENER INDEX (USED TO EVALUATE DIVERSITY FROM THE FORMULA OF THE SHANNON ENTROPY)

 The Shannon-Wiener Index in the context of religious centers, this index
 quantifies the diversity of religious affiliations within a particular area
 or municipality.

 The formula for the Shannon-Wiener Index is:
 H' = -Σ (p_i * log(p_i))
 where:
   - H' is the Shannon-Wiener Index,
   - p_i is the proportion of the i-th religion (i.e., the proportion of religious centers 
     belonging to religion 'i' in the municipality),
   - log(p_i) is the natural logarithm of the proportion p_i.

 The Shannon-Wiener Index ranges from 0 to ln(S), where S is the number of religions considered.
 A value of 0 indicates no diversity (i.e., only one religion is present), while higher values 
 indicate greater diversity (i.e., a more even distribution of religious centers across multiple religions).

 In this analysis, the Shannon-Wiener Index is used to assess the diversity of religious centers
 in different municipalities. By calculating this index for each municipality, we can compare 
 how the distribution of religious centers varies and how this diversity relates to other factors,
 such as the proportion of foreign-born population in the municipality.

 A higher Shannon-Wiener Index value suggests that the religious landscape is more diverse, 
 while a lower value suggests a more homogenous religious environment.


In [44]:
# Número de religiones (columnas de religiones)
num_religions = len(religions)

# Calcular las proporciones por municipio (como antes)
for religion in religions:
    foreign_faith_distribution[religion + '_proportion'] = foreign_faith_distribution[religion] / foreign_faith_distribution['Total_2021']

# Función para calcular el índice de Shannon-Wiener
def shannon_wiener_index(row, religions):
    return -np.sum([p * np.log(p) for p in row[religions] if p > 0])

# Función para normalizar el índice
def normalized_shannon_index(row, religions, num_religions):
    shannon_index = shannon_wiener_index(row, religions)
    return shannon_index / np.log(num_religions)

# Aplicar la función para calcular el índice normalizado
foreign_faith_distribution['Normalized_Shaw_Idx'] = foreign_faith_distribution.apply(
    normalized_shannon_index, axis=1, religions=[religion + '_proportion' for religion in religions], num_religions=num_religions
)

# Z-score normalization for the Shannon-Wiener Index (Normalized Shannon Index)
foreign_faith_distribution['Zscore_Shannon_2021'] = (
    foreign_faith_distribution['Normalized_Shaw_Idx'] - foreign_faith_distribution['Normalized_Shaw_Idx'].mean()
) / foreign_faith_distribution['Normalized_Shaw_Idx'].std()


## SAVE THE GEODATAFRAME

In [45]:
foreign_faith_distribution.to_file("foreign_faith_distribution.json")